# Call an REST API published via API Management

Make sure you have create an API Management instance. If you need help, follow the Getting Started guide here: [Create a new Azure API Management service instance by using the Azure portal](https://learn.microsoft.com/en-us/azure/api-management/get-started-create-service-instance)

As an example of an API not residing in Azure that you can call, you may use the [NIST Certified cryptographically secure random number generator](https://csrng.net/) API, that are available here: https://csrng.net/csrng/csrng.php

For example: https://csrng.net/csrng/csrng.php?min=0&max=100

will return the something like:

```
[{"status":"success","min":0,"max":100,"random":99}]
```

The HTTPS REST API is not in an domain that is allowed for `sp_invoke_external_rest_endpoint` usage. See the allowed list of endpoints here: [Allowed Endpoints](https://learn.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-invoke-external-rest-endpoint-transact-sql?view=azuresqldb-current&tabs=request-headers#allowed-endpoints)

In order to make the NIST endpoint available to be used by `sp_invoke_external_rest_endpoint`, it must published using Azure API Management.

In the next samples is assumed that the API Management instance is deployed at https://external.azure-api.net and that the NIST API has been published at the `/random` path. 

Once the API is published, get the Subscription key you can use to call the created API and store it into a Database Scoped Credential


In [ ]:
-- make sure a database master key exists
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##') begin
    create master key encryption by password = 'LONg_Pa$$_w0rd!'
end

-- create database scoped credential
if exists(select * from sys.database_scoped_credentials where [name] = 'https://api-mgmt.azure-api.net/random') begin
    drop database scoped credential [https://api-mgmt.azure-api.net/random];
end
create database scoped credential [https://api-mgmt.azure-api.net/random]
with identity = 'HTTPEndpointHeaders', secret = '{"Ocp-Apim-Subscription-Key":"***"}';
go

And then call random number generator via the API Management endpoint:

In [ ]:
declare @ret int, @response nvarchar(max);
exec @ret = sp_invoke_external_rest_endpoint 
    @url = N'https://api-mgmt.azure-api.net/random?min=0&max=100',
    @method = 'GET', 
    @credential = [https://api-mgmt.azure-api.net/random],
    @response = @response OUTPUT
select 
    @ret as ReturnCode,     
    json_query(@response, '$.response') as Response,
    json_query(@response, '$.result') as Result;

